In [1]:
import global_values as GV

In [2]:
GV.debug = False

In [3]:
import pandas as pd
from src.model.mula_tabpro import MultiAgentDataPrep
from src.data import TQAData

In [4]:
model = MultiAgentDataPrep()

In [5]:
# save a table with dirty data to df
df = pd.DataFrame().from_dict({
    'name': ['John', 'Jane', 'Jim', 'Jill', 'Jack'],
    'age': [20, 30, 40, 50, 60],
    'salary': ['$2000', '$3000', '$4000', '$5000', '$6000'],
    'city': ['New York', 'Los Angeles', 'New York', 'Houston', 'Los Angeles'],
})
df

,name,age,salary,city
0,John,20,$2000,New York
1,Jane,30,$3000,Los Angeles
2,Jim,40,$4000,New York
3,Jill,50,$5000,Houston
4,Jack,60,$6000,Los Angeles


In [6]:
question = 'What is the average salary of the employees who from New York and Los Angeles?'
title = 'Salary Sheet'
d = TQAData(dataset_name='null', tbl=df, question=question, title=title, label='null')
d

In [7]:
GV.debug

False

In [8]:
logical_plan, binder_sql = model.generate_logical_plan(d)

False
Column Headers Cleaning Report:


HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


In [9]:
logical_plan[0].req

'remove dollar sign and commas'

In [10]:
logical_plan[1].rel_cols

['city']

In [13]:
physical_plan = model.generate_physical_plan(d, logical_plan, binder_sql)

HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


In [14]:
data = model.execute_physical_plan(d, physical_plan)

In [15]:
data.tbl


,city,salary_cleaned
0,new york,2000
1,los angeles,3000
2,new york,4000
3,houston,5000
4,los angeles,6000
